# Wage Prediction using Machine Learning

In this notebook we will answer what determines the wage of workers from a predictive perspective.

This example focuses on a sample of Registered Nurses in the US collected during 2017. The hourly wage of a nurse is denoted by $Y$ and $X$ is a vector of nurses' characteristics, e.g., human capital, demographics, job-relevant characteristics. The question that we want to answer is:

- How to use nurses' characteristics, such as education and experience, to best predict wages?


In [1]:
import numpy as np
import pandas as pd
import patsy
from sklearn.model_selection import train_test_split, KFold
from sklearn.preprocessing import StandardScaler
from sklearn.linear_model import LassoCV, RidgeCV, ElasticNetCV, LinearRegression
from sklearn.tree import DecisionTreeRegressor
from sklearn.ensemble import RandomForestRegressor, GradientBoostingRegressor, StackingRegressor
from sklearn.neural_network import MLPRegressor
from sklearn.base import TransformerMixin, RegressorMixin, BaseEstimator
from sklearn.pipeline import make_pipeline

import warnings
warnings.simplefilter('ignore')

## Data

In [2]:
# Read data
df = pd.read_csv('data/wages_nurses.csv')
df.tail()

,lwage,female,age,race,children,marital,education,RN_experience,left_nursing,country_ed_US,english_only,military,certificates,labor_union,work_setting,work_situation,level_care,care_specialty,state
5617,3.496087,1,72,white,0,widowed_divorced_sep,ed_assoc,51,left_0,1,1,never_served_mil,0,0,SET_long_term_inpatient,SIT_self_employed,LC_nursing_home,CC_no_patient_care,state_n13
5618,3.678408,1,50,white,0,never_married,ed_msn,14,left_0,1,1,never_served_mil,0,0,SET_hospital,SIT_agency_facility,LC_education,CC_emergency_care,state_n24
5619,3.952845,0,36,white,0,currently_married,ed_bsn,4,left_0,1,1,never_served_mil,2,1,SET_hospital,SIT_agency_facility,LC_inpatient,CC_oncology,state_n32
5620,3.957960,1,61,white,0,currently_married,ed_bsn,40,left_0,1,1,never_served_mil,0,0,SET_hospital,SIT_agency_facility,LC_inpatient,CC_no_patient_care,state_n37
5621,3.100270,1,38,white,0,currently_married,ed_assoc,3,left_0,1,1,never_served_mil,2,0,SET_hospital,SIT_agency_facility,LC_others,CC_medical_surgical,state_n29


In [3]:
# Output variable
y = np.log(df['lwage']).values
Z = df.drop(['lwage'], axis=1)
Z.shape

(5622, 18)

Construct a prediction rule for hourly (log) wage $y$, which depends linearly on relevant characteristics $Z$:

$$
y = g(Z) + e
$$

Then, assess the predictive performance of a given model using the (adjusted) sample MSE, the (adjusted) sample $R^2$ and the out-of-sample MSE and $R^2$. Thus, we measure the prediction quality of the models via data splitting.


In [4]:
train_idx, test_idx = train_test_split(np.arange(len(y)), test_size=0.20, random_state=42)
y_train, y_test = y[train_idx], y[test_idx]

## Linear Models

We employ two different specifications for prediction:

1. Basic Model: $X$ consists of a set of raw regressors

2. Flexible Model: $X$ consists of all raw regressors from the basic model plus a dictionary of transformations.


In [5]:
model_base = ('0 + female + age + race + children + marital '
              '+ education + RN_experience + RN_experience**2 + RN_experience**3 + RN_experience**4 '
              '+ left_nursing + country_ed_US + english_only + military + certificates'
              '+ labor_union + work_setting + work_situation + level_care + care_specialty + state')
Zbase = patsy.dmatrix(model_base, Z, return_type='dataframe').values
X_train, X_test = Zbase[train_idx], Zbase[test_idx]

In [6]:
def metrics(X_test, y_test, estimator):
    mse = np.mean((y_test - estimator.predict(X_test))**2)
    semse = np.std((y_test - estimator.predict(X_test))**2) / np.sqrt(len(y_test))
    r2 = 1 - mse / np.var(y_test)
    print(f'{mse:.4f}, {semse:.4f}, {r2:.4f}')
    return mse, semse, r2

results = {}

Let's start by running a simple OLS regression.

In [7]:
# Basic model
lr = LinearRegression().fit(X_train, y_train)
ypred_ols = lr.predict(X_test)
results['ols'] = metrics(X_test, y_test, lr)

0.0086, 0.0007, 0.2342


In [8]:
cv = KFold(n_splits=5, shuffle=True, random_state=42)

# Lasso
lcv = make_pipeline(StandardScaler(), LassoCV(cv=cv, random_state=42)).fit(X_train, y_train)
ypred_lcv = lcv.predict(X_test)
results['lassocv'] = metrics(X_test, y_test, lcv)

0.0087, 0.0007, 0.2328


In [9]:
# Ridge
rcv = make_pipeline(StandardScaler(), RidgeCV(cv=cv)).fit(X_train, y_train)
ypred_rcv = rcv.predict(X_test)
results['ridgecv'] = metrics(X_test, y_test, rcv)

0.0086, 0.0007, 0.2343


In [10]:
# Elastic Net
ecv = make_pipeline(StandardScaler(), ElasticNetCV(cv=cv, random_state=42)).fit(X_train, y_train)
ypred_ecv = ecv.predict(X_test)
results['elanetcv'] = metrics(X_test, y_test, ecv)

0.0087, 0.0007, 0.2330


Use theoretical-optimal $\lambda$ for lasso. 

This is a based on a Python implementation made by [Max Huppertz](https://maxhuppertz.github.io/code/). His library is this [repository](https://github.com/maxhuppertz/hdmpy).

Run the following code to install the library:

```python
!git clone https://github.com/maxhuppertz/hdmpy.git
!pip install multiprocess
```

In [11]:
import hdmpy

# Wrap the package so that it has the familiar sklearn API
class RLasso(BaseEstimator, RegressorMixin):

    def __init__(self, *, post=True):
        self.post = post

    def fit(self, X, y):
        self.rlasso_ = hdmpy.rlasso(X, y, post=self.post)
        return self

    @property
    def coef_(self):
        return np.array(self.rlasso_.est['beta']).flatten()

    @property
    def intercept_(self):
        return np.array(self.rlasso_.est['intercept'])

    def predict(self, X):
        return X @ self.coef_ + self.intercept_

In [12]:
# Optimal lasso
lasso = make_pipeline(StandardScaler(), RLasso(post=False)).fit(X_train, y_train)
ypred_lasso = lasso.predict(X_test)
results['lasso'] = metrics(X_test, y_test, lasso)

0.0090, 0.0007, 0.2019


In [13]:
# Optimal post-lasso
postlasso = make_pipeline(StandardScaler(), RLasso(post=True)).fit(X_train, y_train)
ypred_postlasso = postlasso.predict(X_test)
results['postlasso'] = metrics(X_test, y_test, postlasso)

0.0091, 0.0007, 0.1977


## Non-Linear Models

Apply regression trees, random forests, boosted trees and neural nets to estimate the regression function $g(Z)$.

In [14]:
dtr = DecisionTreeRegressor(ccp_alpha=0.001, min_samples_leaf=5, random_state=42).fit(X_train, y_train)
ypred_dtr = dtr.predict(X_test)
results['trees'] = metrics(X_test, y_test, dtr)

0.0113, 0.0008, -0.0005


In [15]:
rf = RandomForestRegressor(n_estimators=2000, min_samples_leaf=5, random_state=42)
rf.fit(X_train, y_train)
ypred_rf = rf.predict(X_test)
results['randforest'] = metrics(X_test, y_test, rf)

0.0089, 0.0007, 0.2126


In [16]:
gbf = GradientBoostingRegressor(n_estimators=1000, learning_rate=.01,
                                subsample=.5, max_depth=2, random_state=42)
gbf.fit(X_train, y_train)
ypred_gbf = gbf.predict(X_test)
results['gradboost'] = metrics(X_test, y_test, gbf)

0.0088, 0.0007, 0.2162


In [17]:
nnet = MLPRegressor((200, 20,), 'relu',
                    learning_rate_init=0.01, batch_size=10, max_iter=10, random_state=42)
nnet.fit(X_train, y_train)
ypred_nnet = nnet.predict(X_test)
results['neuralnet'] = metrics(X_test, y_test, nnet)

0.0089, 0.0007, 0.2143


In [18]:
import skorch
from torch import nn, optim

arch = nn.Sequential(nn.Linear(X_train.shape[1], 200), nn.ReLU(),
                     nn.Linear(200, 20), nn.ReLU(),
                     nn.Linear(20, 1))
nnet_early = skorch.NeuralNetRegressor(arch, lr=0.01, batch_size=10,
                                       max_epochs=100,
                                       optimizer=optim.Adam,
                                       callbacks=[skorch.callbacks.EarlyStopping()])
nnet_early.fit(X_train.astype(np.float32), y_train.reshape(-1, 1).astype(np.float32))
ypred_nnet_early = nnet_early.predict(X_test.astype(np.float32)).flatten()
results['neuralnet_early'] = metrics(X_test.astype(np.float32),
                                y_test.reshape(-1, 1).astype(np.float32), nnet_early)

  epoch    train_loss    valid_loss     dur
-------  ------------  ------------  ------
      1        0.0496        0.0161  0.2114
      2        0.0146        0.0132  0.1843
      3        0.0111        0.0114  0.1822
      4        0.0100        0.0129  0.1814
      5        0.0095        0.0099  0.1836
      6        0.0088        0.0099  0.1820
      7        0.0083        0.0097  0.1814
      8        0.0081        0.0084  0.1814
      9        0.0080        0.0079  0.1814
     10        0.0079        0.0081  0.1829
     11        0.0077        0.0084  0.1809
     12        0.0078        0.0090  0.1816
     13        0.0078        0.0098  0.1817
Stopping since valid_loss has not improved in the last 5 epochs.
0.0102, 0.0007, 0.0951


In [19]:
df = pd.DataFrame(results).T
df.columns = ['MSE', 'S.E. MSE', 'Rsq']
df.sort_values('MSE', ascending=True)

,MSE,S.E. MSE,Rsq
ridgecv,0.008646,0.000701,0.234312
ols,0.008647,0.000701,0.234199
elanetcv,0.008661,0.000703,0.232957
lassocv,0.008663,0.000704,0.232775
gradboost,0.008850,0.000694,0.216247
neuralnet,0.008872,0.000718,0.214316
randforest,0.008891,0.000667,0.212566
lasso,0.009012,0.000728,0.201858
postlasso,0.009059,0.000723,0.197714
neuralnet_early,0.010218,0.000749,0.095105


## Stacking

In the final step, build a prediction model by combining the strength of each model.
$$
g(z) = \sum_{k=1}^K \alpha_k g_k(z)
$$
where $g_k$'s denote our prediction rules from the table above and the $\alpha_k$'s are the corresponding weights.

In [20]:
method_name = ['OLS', 'CV Lasso', 'CV Ridge', 'CV ElasticNet', 'Lasso', 'Post-Lasso OLS',
               'Decision Tree', 'Random Forest', 'Boosted Forest', 'Neural Net', 'Neural Net (early stopping)']
ypreds = np.stack((ypred_ols, ypred_lcv, ypred_rcv, ypred_ecv, ypred_lasso, ypred_postlasso,
                   ypred_dtr, ypred_rf, ypred_gbf, ypred_nnet, ypred_nnet_early), axis=-1)
stack_ols = LinearRegression().fit(ypreds, y_test)
pd.DataFrame({'weight': stack_ols.coef_}, index=method_name).round(2)

,weight
OLS,2.77
CV Lasso,-21.29
CV Ridge,-2.56
CV ElasticNet,21.59
Lasso,-0.69
Post-Lasso OLS,0.38
Decision Tree,4.17
Random Forest,0.42
Boosted Forest,0.14
Neural Net,0.32


In [21]:
# Calculate the test sample MSE and Rsq
# We should have left out a third sample to validate the performance of the stacked model
mse = np.mean((y_test - stack_ols.predict(ypreds))**2)
r2 = 1 - mse / np.var(y_test)
print(f'MSE: {mse:.4f}, R^2: {r2:.4f}')

MSE: 0.0083, R^2: 0.2665


## Pipelines

We can also do it in a more sklearn way, by defining a formula transformer and corresponding pipelines

In [22]:
class FormulaTransformer(TransformerMixin, BaseEstimator):

    def __init__(self, formula):
        self.formula = formula

    def fit(self, X, y=None):
        mat = patsy.dmatrix(self.formula, X, return_type='matrix')
        self.design_info = mat.design_info
        return self

    def transform(self, X, y=None):
        return patsy.build_design_matrices([self.design_info], X)[0]

In [23]:
base = FormulaTransformer(model_base)
methods = [('ols', make_pipeline(base, LinearRegression())),
           ('lasso', make_pipeline(base, StandardScaler(), RLasso(post=False))),
           ('postlasso', make_pipeline(base, StandardScaler(), RLasso(post=True))),
           ('lcv', make_pipeline(base, StandardScaler(), LassoCV())),
           ('rcv', make_pipeline(base, StandardScaler(), RidgeCV())),
           ('ecv', make_pipeline(base, StandardScaler(), ElasticNetCV())),
           ('dtr', make_pipeline(base, DecisionTreeRegressor(ccp_alpha=0.001, min_samples_leaf=5,
                                                             random_state=123))),
           ('rf', make_pipeline(base, RandomForestRegressor(n_estimators=2000, min_samples_leaf=5,
                                                            random_state=123))),
           ('gbf', make_pipeline(base, GradientBoostingRegressor(n_estimators=1000, learning_rate=.01,
                                                                 subsample=.5, max_depth=2,
                                                                 random_state=123))),
           ('nnet', make_pipeline(base, MLPRegressor((200, 20,), 'relu',
                                                     learning_rate_init=0.01,
                                                     batch_size=10, max_iter=10,
                                                     random_state=123)))]

In [24]:
results = {}
ypreds = np.zeros((len(test_idx), len(methods)))  # test predictions used for stacking

for it, (name, estimator) in enumerate(methods):
    estimator.fit(Z.iloc[train_idx], y[train_idx])
    results[name] = metrics(Z.iloc[test_idx], y[test_idx], estimator)
    ypreds[:, it] = estimator.predict(Z.iloc[test_idx])

0.0086, 0.0007, 0.2342
0.0090, 0.0007, 0.2019
0.0091, 0.0007, 0.1977
0.0087, 0.0007, 0.2333
0.0086, 0.0007, 0.2343
0.0087, 0.0007, 0.2333
0.0113, 0.0008, -0.0005
0.0089, 0.0007, 0.2126
0.0088, 0.0007, 0.2226
0.0099, 0.0007, 0.1273


In [25]:
df = pd.DataFrame(results).T
df.columns = ['MSE', 'S.E. MSE', 'Rsq']
df

,MSE,S.E. MSE,Rsq
ols,0.008647,0.000701,0.234199
lasso,0.009012,0.000728,0.201858
postlasso,0.009059,0.000723,0.197714
lcv,0.008657,0.000703,0.233345
rcv,0.008646,0.000701,0.234312
ecv,0.008657,0.000703,0.233332
dtr,0.011298,0.000780,-0.000550
rf,0.008892,0.000667,0.212551
gbf,0.008779,0.000688,0.222555
nnet,0.009854,0.000746,0.127279


1. Partition the data in k-folds

2. For each fold, train each of the estimators in the `methods` parameter on all the data outside of the fold and then predict on the data in the fold

3. Train a `final_estimator` predicting the true outcome using the out-of-fold predictions of each method as features

4. All the base estimators are re-fitted on all the data and the final predictor will first predict based on each fitted based estimator and then aggregate based on the fitted `final_estimator`


In [26]:
# Stacking with sklearn
stack = StackingRegressor(methods, final_estimator=RLasso(), cv=3, verbose=3)
stack.fit(Z.iloc[train_idx], y[train_idx])

StackingRegressor(cv=3,
                  estimators=[('ols',
                               Pipeline(steps=[('formulatransformer',
                                                FormulaTransformer(formula='0 '
                                                                           '+ '
                                                                           'female '
                                                                           '+ '
                                                                           'age '
                                                                           '+ '
                                                                           'race '
                                                                           '+ '
                                                                           'children '
                                                                           '+ '
                                                                           'marital '
                                                                           '+ '
                                                                           'education '
                                                                           '+ '
                                                                           'RN_experience '
                                                                           '+ '
                                                                           'RN_experience**2 '
                                                                           '+ '
                                                                           'RN_experience**3 '
                                                                           '+ '
                                                                           'RN_experience**4 '
                                                                           '+ '
                                                                           'left_nursing '
                                                                           '+ '
                                                                           'country_ed_US '
                                                                           '+ '
                                                                           'english_only '
                                                                           '+ '
                                                                           'military '
                                                                           '+ '
                                                                           'certificates+ '
                                                                           'labor_un...
                                                                           'RN_experience**4 '
                                                                           '+ '
                                                                           'left_nursing '
                                                                           '+ '
                                                                           'country_ed_US '
                                                                           '+ '
                                                                           'english_only '
                                                                           '+ '
                                                                           'military '
                                                                           '+ '
                                                                           'certificates+ '
                                                                           'labor_union '
                                                                           '+ '
                                         

In [27]:
# Weights of the final estimator
pd.DataFrame({'weight': stack.final_estimator_.coef_}, index=[name for name, _ in methods])

,weight
ols,1.767026
lasso,0.000000
postlasso,0.000000
lcv,0.000000
rcv,-1.205465
ecv,0.000000
dtr,0.000000
rf,0.301635
gbf,0.191599
nnet,0.000000


In [28]:
# Out-of-sample performance
mse, semse, r2 = metrics(Z.iloc[test_idx], y[test_idx], stack)

0.0084, 0.0007, 0.2579


## AutoML

In [29]:
from flaml import AutoML

automl = make_pipeline(base, AutoML(task='regression', time_budget=60, early_stop=True,
                                    eval_method='cv', n_splits=3, metric='r2', verbose=3,))

automl.fit(Z.iloc[train_idx], y[train_idx])

[flaml.automl.logger: 07-31 17:25:22] {1680} INFO - task = regression
[flaml.automl.logger: 07-31 17:25:22] {1691} INFO - Evaluation method: cv
[flaml.automl.logger: 07-31 17:25:22] {1789} INFO - Minimizing error metric: 1-r2
[flaml.automl.logger: 07-31 17:25:22] {1901} INFO - List of ML learners in AutoML Run: ['lgbm', 'rf', 'xgboost', 'extra_tree', 'xgb_limitdepth']
[flaml.automl.logger: 07-31 17:25:22] {2219} INFO - iteration 0, current learner lgbm
[flaml.automl.logger: 07-31 17:25:22] {2345} INFO - Estimated sufficient time budget=384s. Estimated necessary time budget=3s.
[flaml.automl.logger: 07-31 17:25:22] {2392} INFO -  at 0.0s,	estimator lgbm's best error=0.9308,	best estimator lgbm's best error=0.9308
[flaml.automl.logger: 07-31 17:25:22] {2219} INFO - iteration 1, current learner lgbm
[flaml.automl.logger: 07-31 17:25:22] {2392} INFO -  at 0.1s,	estimator lgbm's best error=0.9308,	best estimator lgbm's best error=0.9308
[flaml.automl.logger: 07-31 17:25:22] {2219} INFO - it

Pipeline(steps=[('formulatransformer',
                 FormulaTransformer(formula='0 + female + age + race + '
                                            'children + marital + education + '
                                            'RN_experience + RN_experience**2 '
                                            '+ RN_experience**3 + '
                                            'RN_experience**4 + left_nursing + '
                                            'country_ed_US + english_only + '
                                            'military + certificates+ '
                                            'labor_union + work_setting + '
                                            'work_situation + level_care + '
                                            'care_specialty + state')),
                ('automl',
                 AutoML(ap...
                        keep_search_state=False, learner_selector='sample',
                        log_file_name='', log_training_metric=False,
                        log_type='better', max_iter=None, mem_thres=4294967296,
                        metric='r2', metric_constraints=[],
                        min_sample_size=10000, mlflow_logging=True,
                        model_history=False, n_concurrent_trials=1, n_jobs=-1,
                        n_splits=3, pred_time_limit=inf,
                        preserve_checkpoint=True, retrain_full=True, ...))])

In [30]:
# Out-of-sample performance
mse, semse, r2 = metrics(Z.iloc[test_idx], y[test_idx], automl)

0.0085, 0.0007, 0.2451


In [31]:
# Stacking with flaml
automl = make_pipeline(base, AutoML(task='regression', time_budget=60, early_stop=True,
                                    eval_method='cv', n_splits=3, metric='r2', verbose=3,
                                    ensemble={'passthrough': False,  # stacker will use raw X's instead of predictions
                                              'final_estimator': RLasso()}))

automl.fit(Z.iloc[train_idx], y[train_idx])

[flaml.automl.logger: 07-31 17:26:22] {1680} INFO - task = regression
[flaml.automl.logger: 07-31 17:26:22] {1691} INFO - Evaluation method: cv
[flaml.automl.logger: 07-31 17:26:22] {1789} INFO - Minimizing error metric: 1-r2
[flaml.automl.logger: 07-31 17:26:22] {1901} INFO - List of ML learners in AutoML Run: ['lgbm', 'rf', 'xgboost', 'extra_tree', 'xgb_limitdepth']
[flaml.automl.logger: 07-31 17:26:22] {2219} INFO - iteration 0, current learner lgbm
[flaml.automl.logger: 07-31 17:26:22] {2345} INFO - Estimated sufficient time budget=369s. Estimated necessary time budget=3s.
[flaml.automl.logger: 07-31 17:26:22] {2392} INFO -  at 0.0s,	estimator lgbm's best error=0.9308,	best estimator lgbm's best error=0.9308
[flaml.automl.logger: 07-31 17:26:22] {2219} INFO - iteration 1, current learner lgbm
[flaml.automl.logger: 07-31 17:26:22] {2392} INFO -  at 0.1s,	estimator lgbm's best error=0.9308,	best estimator lgbm's best error=0.9308
[flaml.automl.logger: 07-31 17:26:22] {2219} INFO - it

Pipeline(steps=[('formulatransformer',
                 FormulaTransformer(formula='0 + female + age + race + '
                                            'children + marital + education + '
                                            'RN_experience + RN_experience**2 '
                                            '+ RN_experience**3 + '
                                            'RN_experience**4 + left_nursing + '
                                            'country_ed_US + english_only + '
                                            'military + certificates+ '
                                            'labor_union + work_setting + '
                                            'work_situation + level_care + '
                                            'care_specialty + state')),
                ('automl',
                 AutoML(ap...
                        keep_search_state=False, learner_selector='sample',
                        log_file_name='', log_training_metric=False,
                        log_type='better', max_iter=None, mem_thres=4294967296,
                        metric='r2', metric_constraints=[],
                        min_sample_size=10000, mlflow_logging=True,
                        model_history=False, n_concurrent_trials=1, n_jobs=-1,
                        n_splits=3, pred_time_limit=inf,
                        preserve_checkpoint=True, retrain_full=True, ...))])

In [32]:
# Out-of-sample performance
mse, semse, r2 = metrics(Z.iloc[test_idx], y[test_idx], automl)

0.0084, 0.0007, 0.2555
